In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from scipy.io import loadmat
from torch.utils.data import DataLoader, Dataset

In [3]:
!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat

--2021-04-15 09:54:25--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  12.5MB/s    in 14s     

2021-04-15 09:54:39 (12.3 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2021-04-15 09:54:39--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  14.8MB/s    in 5.0s    

2021-04-15 09:54:44 (12.2 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [2]:
class SVHN_dataset(Dataset):

    def __init__(self, data):
        # Transform data to Torch Tensors
        self.images = torch.tensor(data['X']).permute([3,2,0,1])
        self.labels = torch.tensor(data['y'])
        self.size = self.labels.shape[0]

        # replace label 10 with label 0
        self.labels[self.labels==10] = 0
        # convert to float and normalize images to 0..1 range
        self.images = torch.FloatTensor(self.images/255.)

    def __len__(self):
        return self.size

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [3]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(1,6,(5,5),padding=2,stride=1),
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(6,12,(3,3),padding=1,stride=1),
        nn.BatchNorm2d(12),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(12,24,(3,3),padding=1,stride=1),
        nn.BatchNorm2d(24),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.fc = nn.Linear(512,10)

  def forward(self, x):
    out = self.layer1(x)
    print(out.shape)

    out = self.layer2(out)
    print(out.shape)

    out = self.layer3(out)
    print(out.shape)

    out = out.reshape(out.size(0),-1)
    print(out.shape)

    out = self.fc(out)
    print(out.shape)

    return out

In [4]:
train = loadmat('train_32x32.mat')
test = loadmat('test_32x32.mat')

In [7]:
print(train['X'].shape)

(32, 32, 3, 73257)


In [5]:
train_SVHNdataset = SVHN_dataset(data = train)
test_SVHNdataset = SVHN_dataset(data = test)

In [6]:
model = CNN()
device = torch.device("cuda:0" if torch.cuda.is_available() else cpu)
print(device)

cuda:0


In [7]:
model.to(device)

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=512, out_features=10, bias=True)
)

In [21]:
X_train_tensor = train_SVHNdataset.images.to(device)
x_test_tensor = train_SVHNdataset.labels.to(device)

Y_train_tensor = test_SVHNdataset.images.to(device)
y_test_tensor = train_SVHNdataset.labels.to(device)

In [28]:
Y_train_tensor.shape

torch.Size([26032, 3, 32, 32])

In [22]:
learning_rate = 0.001
lossfunction = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [23]:
num_epochs = 10
loss_values = list()
for epoch in range(1, num_epochs):
  outputs = model(X_train_tensor)
  loss = lossfunction(outputs, Y_train_tensor)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  print('Epoch - %d, loss - 0.5f ',epoch, loss.item())
  loss_values.append(loss.item())